In [4]:
!pip install PyPDF2

     -------------------------------------- 232.6/232.6 kB 7.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import email
from email import policy
from email.parser import BytesParser
import PyPDF2
from io import BytesIO

def extract_pdf_text_from_eml(eml_file_path):
    result = {
        'email_body': None,  # Email body content
        'pdf_attachments': []  # List of PDF attachments
    }

    try:
        # Read and parse the .eml file
        with open(eml_file_path, 'rb') as f:
            msg = BytesParser(policy=policy.default).parse(f)

        # Check if the email has multiple parts (multipart)
        if msg.is_multipart():
            # Iterate through each part of the email
            for part in msg.iter_parts():
                content_disposition = part.get('Content-Disposition')
                # Check if it's the main email body (text/plain or text/html)
                if part.get_content_type() == 'text/plain' and not content_disposition:
                    result['email_body'] = part.get_payload(decode=True).decode(part.get_content_charset())
                # Check if the part is an attachment
                if content_disposition and 'attachment' in content_disposition:
                    filename = part.get_filename()
                    content_type = part.get_content_type()
                    # Process the PDF attachment
                    if content_type == 'application/pdf':
                        # Extract the PDF content as raw binary
                        pdf_data = part.get_payload(decode=True)

                        # Use PyPDF2 to read the PDF
                        pdf_text = extract_text_from_pdf(pdf_data)

                        # Store the PDF text in the result dictionary
                        result['pdf_attachments'].append({
                            'filename': filename,
                            'content': pdf_text
                        })

        return result

    except Exception as e:
        print(f"Error while extracting PDF content: {e}")
        return {'error': str(e)}

def extract_text_from_pdf(pdf_data):
    # Convert binary PDF data to a BytesIO object
    pdf_file = BytesIO(pdf_data)
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Extract text from all pages of the PDF
    pdf_text = ''
    for page in pdf_reader.pages:
        pdf_text += page.extract_text()

    return pdf_text

# Example usage
eml_file_path = 'emails/Proposal for Loan Restructuring and Compliance Documentation.eml'  # Path to your .eml file
result = extract_pdf_text_from_eml(eml_file_path)

# Print the result (email body and PDF text)
#print("Email Body Content:")
#print(result.get('email_body', 'No email body found.'))
email_content = result.get('email_body')
attachment_content=''
if result['pdf_attachments']:
    #print("\nPDF Attachment Content:")
    for pdf in result['pdf_attachments']:
       # print(f"--- {pdf['filename']} ---")
       # print(pdf['content'])  # Print PDF text content
        attachment_content+=pdf['content']
        


In [22]:
#Create the dictionary object to send both the mail and attachment content
completeMailContent = {
    "emailContent":email_content,
    "attachmentContent":attachment_content
}
payload = {"content": completeMailContent}
#print(payload)

In [23]:
import requests
#Endpoint for the email segregation
api_url = "http://localhost:8080/getEmailContent" 

#send post request
response = requests.post(api_url, json=payload)

# Print response
print(response.status_code)
print(response.text)